In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_csv('/Users/maxi/Downloads/SensorenMitVollständigenDaten2019-2023.csv')
df_Kreis_code = pd.read_csv('/Users/maxi/Downloads/nrw-postleitzahlen.csv', sep=';') 

/var/folders/c0/bb24fmsx45vgrsz0nl893g940000gn/T/ipykernel_48717/3166022728.py:1: DtypeWarning: Columns (10,30,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Users/maxi/Downloads/SensorenMitVollständigenDaten2019-2023.csv')


In [3]:
df_Geo = pd.read_csv(
    "/Users/maxi/Downloads/INKAR - Indikatoren und Karten zur Raum- und Stadtentwicklung.csv",
    sep=";",
    header=[0, 1],
    quotechar='"',
    encoding="utf-8",
    low_memory=False
)

df_Geo.columns = [
    f"{oben}_{unten}" if pd.notna(unten) and unten != "" else oben
    for oben, unten in df_Geo.columns
]

num_cols = [c for c in df_Geo.columns if "_" in c]

df_Geo[num_cols] = (
    df_Geo[num_cols]                 # <<< hier df_Geo, nicht df
        .replace({r"\.": "", ",": "."}, regex=True)
        .apply(pd.to_numeric, errors="coerce")
)

In [4]:
df['post_code'] = df['post_code'].astype(int).astype(str).str.zfill(5)
df_Kreis_code['Name'] = df_Kreis_code['Name'].astype(int).astype(str).str.zfill(5)


In [5]:
df_Merge1 = pd.merge(
    df,
    df_Kreis_code,
    left_on='post_code',
    right_on='Name',
    how='left'
)

In [6]:
df_Merge1 = df_Merge1.drop(columns=[
    'construction_year',
    'country',
    'Name',
    'usage_detail', 
    'usage',
    'category',
    'measurement_frequency',
    'measurement_value_type',
    'measurement_type',
    'measurement_unit',
    'measurement_category',
    'Unnamed: 0',
    'PLZ Name (short)', 
    'PLZ Name (long)', 
    'Geometry',
    'Postleitzahl / Post code',
    'Land name',
    'Land code',
    'geo_point_2d'])

In [7]:
df_Geo = df_Geo.rename(columns={
    'Kennziffer_Unnamed: 0_level_1': 'Kreis code'
})

df_Geo = df_Geo.drop(columns=[
    'Raumeinheit_Unnamed: 1_level_1',
    'Aggregat_Unnamed: 2_level_1'
])

In [8]:
id_col = "Kreis code"
value_cols = [c for c in df_Geo.columns if c != id_col]

long = (df_Geo
        .melt(id_vars=id_col,
              value_vars=value_cols,
              var_name="indicator_year",
              value_name="value"))

long[["indicator", "year"]] = long["indicator_year"].str.rsplit("_", n=1, expand=True)
long["year"]  = long["year"].astype(int)
long["value"] = pd.to_numeric(long["value"], errors="coerce")
long = long.drop(columns="indicator_year")

def abs_delta(g):
    g = g.dropna().sort_values("year")
    if len(g) < 2:
        return np.nan
    return (g["value"].iloc[-1] - g["value"].iloc[0]) / (g["year"].iloc[-1] - g["year"].iloc[0])

mean_delta = (
    long.groupby(["indicator", id_col])
        .apply(abs_delta)              # Serie mit MultiIndex
        .loc[lambda s: s != 0]         # reine 0-Trends raus
        .groupby(level=0)
        .mean()                        # Ø Δ je Indikator
        .to_dict()
)

TARGET_YEARS = list(range(2019, 2025))  # 2019-2024 inklusive

def forecast(group):
    kreis = group[id_col].iat[0]
    ind   = group["indicator"].iat[0]

    g = (group.set_index("year")
              .reindex(TARGET_YEARS))
    g[id_col]   = kreis
    g["indicator"] = ind

    known = g["value"].dropna()
    k = len(known)

    if k >= 3:                                    # quadratische Regression
        coeff = np.polyfit(known.index, known.values, 2)
        poly  = np.poly1d(coeff)
        for y in TARGET_YEARS:
            if pd.isna(g.at[y, "value"]):
                g.at[y, "value"] = poly(y)

    elif k == 2:                                  # lineare Regression
        coeff = np.polyfit(known.index, known.values, 1)
        for y in TARGET_YEARS:
            if pd.isna(g.at[y, "value"]):
                g.at[y, "value"] = np.polyval(coeff, y)

    elif k == 1:                                  # nur 1 Punkt → Ø Δ-Drift
        last_year = known.index[0]
        last_val  = known.iloc[0]
        Δ = mean_delta.get(ind, 0.0)
        for y in TARGET_YEARS:
            if pd.isna(g.at[y, "value"]):
                g.at[y, "value"] = last_val + Δ * (y - last_year)


    g["value"] = g["value"].interpolate(limit_direction="both").round(2)
    return g.reset_index()

filled_long = (
    long.groupby([id_col, "indicator"], group_keys=False)
        .apply(forecast)
)

df_Geo_filled = (
    filled_long
      .pivot_table(index=id_col,
                   columns=["indicator", "year"],
                   values="value")
)

df_Geo_filled.columns = [f"{ind}_{yr}" for ind, yr in df_Geo_filled.columns]
df_Geo_filled = df_Geo_filled.reset_index()


In [9]:
df_Merge1["year"] = pd.to_datetime(df_Merge1["Datum_Messung"]).dt.year

id_col = "Kreis code"

pat_year = re.compile(r"_(\d{4})$")
year_cols = [c for c in df_Geo_filled.columns if pat_year.search(c)]

geo_long = (
    df_Geo_filled
      .melt(id_vars=id_col,
            value_vars=year_cols,
            var_name="indicator_year",
            value_name="value")
)

geo_long[["indicator", "year"]] = (
    geo_long["indicator_year"]
           .str.rsplit("_", n=1, expand=True)
)
geo_long["year"]  = geo_long["year"].astype(int)
geo_long = geo_long.drop(columns="indicator_year")

geo_wide = (
    geo_long
      .pivot_table(index=[id_col, "year"],
                   columns="indicator",
                   values="value",
                   aggfunc="first")        # es gibt sowieso nur einen Wert
      .reset_index()
)

geo_wide.columns.name = None   # Kopfzeile entfernen

df_merged = df_Merge1.merge(
    geo_wide,
    on=[id_col, "year"],
    how="left"
)

In [10]:
df_merged.to_csv('/Users/maxi/Desktop/SensorDatenMitGeoDaten.csv', index=False, encoding='utf-8')